Names: Sam Miller, Sami Hakkareinen
Team Name: Iron Hogs
CS135 - Project A

Bag-of-Words Representations
1A: Design Decision Description

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import re


# Step 1: Loading the Data
# Assuming 'x_train.csv' has a column 'text' for the reviews and 'y_train.csv' has a column 'label'
x_train_df = pd.read_csv('/Users/sammiller/Desktop/CS135/projA-release/data_reviews/x_train.csv')
y_train_df = pd.read_csv('/Users/sammiller/Desktop/CS135/projA-release/data_reviews/y_train.csv')

# Step 2: Cleaning the Data
def clean_text(text):
    # Remove punctuation and convert to lowercase
    text = re.sub(r'[^\w\s]', '', text).lower()
    return text

# Apply cleaning function to the text
x_train_df['text'] = x_train_df['text'].apply(clean_text)

# Bag-of-Words Vectorization
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(x_train_df['text'])
y = y_train_df['is_positive_sentiment']




In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Define the Logistic Regression model
log_reg = LogisticRegression(random_state=42)

# Perform cross-validation and print the average accuracy
scores = cross_val_score(log_reg, X, y, cv=5, scoring='roc_auc')
print(f"Average ROC AUC: {scores.mean()}")



from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

log_reg = LogisticRegression(random_state=42)

# Perform cross-validation and print the average ROC AUC
scores = cross_val_score(log_reg, X, y, cv=5, scoring='roc_auc')
print(f"Average ROC AUC: {scores.mean()}")

# Define a pipeline with TF-IDF Vectorizer and Logistic Regression as the classifier for GridSearch
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', LogisticRegression(random_state=42)),
])

# Parameter grid for GridSearch
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'tfidf__max_features': [500, 1000],
    'clf__C': [0.1, 1, 10],
    'clf__max_iter': [100, 200],
}

# Setup GridSearchCV with the pipeline and parameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc', verbose=1)

# Fit the model with the originally processed text and labels
grid_search.fit(x_train_df['text'], y)

# Output the best ROC AUC score and the best parameters
print("Best ROC AUC: ", grid_search.best_score_)
print("Best Parameters: ", grid_search.best_params_)

Average ROC AUC: 0.8705677083333334
Average ROC AUC: 0.8705677083333334
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best ROC AUC:  0.8544079861111111
Best Parameters:  {'clf__C': 1, 'clf__max_iter': 100, 'tfidf__max_features': 1000, 'tfidf__ngram_range': (1, 1)}
